<a href="https://colab.research.google.com/github/vikpy/AISem3/blob/master/CW/ClassWork_Home_Credit_Default_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install kaggle

In [8]:
!mkdir .kaggle

In [9]:
import json
token = {"username":"vikraantpy","key":""}   # Key removed for confidentiality 
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [10]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [11]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [12]:
!chmod 600 /root/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -c home-credit-default-risk -p /content/

 81% 88.0M/109M [00:00<00:00, 142MB/s]
100% 109M/109M [00:00<00:00, 221MB/s] 
 71% 26.0M/36.8M [00:00<00:00, 89.1MB/s]
100% 36.8M/36.8M [00:00<00:00, 120MB/s] 
 95% 257M/271M [00:01<00:00, 181MB/s]
100% 271M/271M [00:01<00:00, 158MB/s]
 44% 16.0M/36.1M [00:00<00:00, 168MB/s]
100% 36.1M/36.1M [00:00<00:00, 177MB/s]
 72% 41.0M/56.8M [00:00<00:00, 67.9MB/s]
100% 56.8M/56.8M [00:00<00:00, 128MB/s] 
 88% 85.0M/96.7M [00:00<00:00, 165MB/s]
100% 96.7M/96.7M [00:00<00:00, 165MB/s]
 80% 61.0M/76.3M [00:00<00:00, 96.4MB/s]
100% 76.3M/76.3M [00:00<00:00, 172MB/s] 
  0% 0.00/524k [00:00<?, ?B/s]
100% 524k/524k [00:00<00:00, 164MB/s]
  0% 0.00/36.5k [00:00<?, ?B/s]
100% 36.5k/36.5k [00:00<00:00, 32.8MB/s]
  0% 0.00/5.81M [00:00<?, ?B/s]
100% 5.81M/5.81M [00:00<00:00, 53.3MB/s]


In [15]:
!unzip \*.zip

Archive:  application_train.csv.zip
  inflating: application_train.csv   

Archive:  installments_payments.csv.zip
  inflating: installments_payments.csv  

Archive:  bureau.csv.zip
  inflating: bureau.csv              

Archive:  application_test.csv.zip
  inflating: application_test.csv    

Archive:  credit_card_balance.csv.zip
  inflating: credit_card_balance.csv  

Archive:  POS_CASH_balance.csv.zip
  inflating: POS_CASH_balance.csv    

Archive:  bureau_balance.csv.zip
  inflating: bureau_balance.csv      

Archive:  previous_application.csv.zip
  inflating: previous_application.csv  

8 archives were successfully processed.


In [16]:
!pip install pyforest 
from pyforest import * 

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13715 sha256=973b5f1a8f4f5190db9c2d00ad08cb99d40bc7ef1881572911e364187f25ef7d
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


# Data Preprocessing 


In [57]:
path_to_contents="/content/"
print("File reading started...")
application_train = pd.read_csv(path_to_contents+"application_train.csv")
application_test = pd.read_csv(path_to_contents+"application_test.csv")
bureau = pd.read_csv(path_to_contents+"bureau.csv")
bureau_balance = pd.read_csv(path_to_contents+"bureau_balance.csv")
credit_card_balance = pd.read_csv(path_to_contents+"credit_card_balance.csv")
installments_payments = pd.read_csv(path_to_contents+"installments_payments.csv")
previous_application = pd.read_csv(path_to_contents+"previous_application.csv")
POS_CASH_balance = pd.read_csv(path_to_contents+"POS_CASH_balance.csv")
print("All files read....")

File reading started...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All files read....


In [58]:
application_test = application_test.reset_index()
application_train = application_train.reset_index()

In [23]:
application_train.shape

(307511, 122)

In [42]:
application_train.isnull().sum()[application_train.isnull().sum()>0].sort_values(ascending=False)

COMMONAREA_MEDI             214865
COMMONAREA_AVG              214865
COMMONAREA_MODE             214865
NONLIVINGAPARTMENTS_AVG     213514
NONLIVINGAPARTMENTS_MODE    213514
                             ...  
EXT_SOURCE_2                   660
AMT_GOODS_PRICE                278
AMT_ANNUITY                     12
CNT_FAM_MEMBERS                  2
DAYS_LAST_PHONE_CHANGE           1
Length: 67, dtype: int64

In [33]:
application_train.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

In [41]:
application_test.isnull().sum()[application_test.isnull().sum()>0].sort_values(ascending=False)

COMMONAREA_MODE             33495
COMMONAREA_AVG              33495
COMMONAREA_MEDI             33495
NONLIVINGAPARTMENTS_MEDI    33347
NONLIVINGAPARTMENTS_AVG     33347
                            ...  
DEF_30_CNT_SOCIAL_CIRCLE       29
OBS_60_CNT_SOCIAL_CIRCLE       29
DEF_60_CNT_SOCIAL_CIRCLE       29
AMT_ANNUITY                    24
EXT_SOURCE_2                    8
Length: 64, dtype: int64

In [43]:
bureau.isnull().sum()[bureau.isnull().sum()>0].sort_values(ascending=False)

AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
dtype: int64

In [39]:
bureau.dtypes

SK_ID_CURR                  int64
SK_ID_BUREAU                int64
CREDIT_ACTIVE              object
CREDIT_CURRENCY            object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
AMT_ANNUITY               float64
dtype: object

In [44]:
bureau.isnull().sum()[bureau.isnull().sum()>0].sort_values(ascending=False)

AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
dtype: int64

In [45]:
bureau_balance.isnull().sum()[bureau_balance.isnull().sum()>0].sort_values(ascending=False)

Series([], dtype: int64)

In [46]:
bureau_balance.dtypes

SK_ID_BUREAU       int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

# We will be using https://www.kaggle.com/volodymyrholomb/xgbmodel-on-base-features method for preprocessing 

In [59]:
application_train.set_index("SK_ID_CURR",
               drop=True,
               inplace=True)

application_test.set_index("SK_ID_CURR",
               drop=True,
               inplace=True)

full_set = pd.concat([application_train, application_test])

bureau_mod = bureau.copy()

closed = (bureau_balance
          .groupby("SK_ID_BUREAU")
          .filter(lambda d: (d["STATUS"] == "C").any())["SK_ID_BUREAU"]
          .unique())

bureau_mod["CREDIT_ACTIVE"] = [v if v != "Active" or i not in closed
                               else "Closed" for v, i in
                               list(zip(bureau_mod["CREDIT_ACTIVE"],
                                        bureau_mod["SK_ID_BUREAU"]))]

(bureau_mod["DAYS_CREDIT_ENDDATE"]
 .fillna(bureau_mod["DAYS_ENDDATE_FACT"],
         inplace=True))

def get_loan_balance(df):
    out = df.loc[(df["CREDIT_ACTIVE"] == "Active") &
                 (df["DAYS_CREDIT_ENDDATE"] > 0), :].copy()

    return ((out["DAYS_CREDIT_ENDDATE"] /
             (out["DAYS_CREDIT"].abs() +
              out["DAYS_CREDIT_ENDDATE"]) *
             out["AMT_CREDIT_SUM"]).sum())

bureau_mod_gr = bureau_mod.groupby("SK_ID_CURR")

bureau_curr = pd.DataFrame(index=application_train.index.tolist() + application_test.index.tolist())

bureau_curr["loan_num"] = bureau_mod_gr.size()

bureau_curr["loan_act"] = (bureau_mod_gr
                           .apply(lambda d:
                                  (d["CREDIT_ACTIVE"] == "Active").sum()))

bureau_curr["loan_ovd"] = (bureau_mod_gr
                           .apply(lambda d:
                                  (d["CREDIT_DAY_OVERDUE"] != 0).sum()))

bureau_curr["loan_bal"] = (bureau_mod_gr
                           .apply(get_loan_balance))

bureau_curr.fillna(0,
                   axis=0,
                   inplace=True)

p_apps_mod = (previous_application
              .loc[(previous_application["SK_ID_CURR"]
                      .isin(full_set.index)), :]
              .query("FLAG_LAST_APPL_PER_CONTRACT == 'Y'\
                      and NFLAG_LAST_APPL_IN_DAY == '1'")
              .copy())

overdued = (installments_payments
            .groupby("SK_ID_PREV")
            .apply(lambda d:
                   (d[d["DAYS_ENTRY_PAYMENT"] > d["DAYS_INSTALMENT"]]
                    .sum())
                   .any())
            .rename("is_overdued"))


p_apps_mod = pd.merge(left=p_apps_mod,
                      right=overdued,
                      how="left",
                      left_on="SK_ID_PREV",
                      right_index=True)

exceeded = (credit_card_balance
            .groupby("SK_ID_PREV")
            .apply(lambda d:
                   (d[d["AMT_BALANCE"] > d["AMT_CREDIT_LIMIT_ACTUAL"]]
                    .astype("bool")
                    .sum())
                   .any())
            .rename("limit_exceeded"))


p_apps_mod = pd.merge(left=p_apps_mod,
                      right=exceeded,
                      how="left",
                      left_on="SK_ID_PREV",
                      right_index=True)



p_apps_mod_gr = p_apps_mod.groupby("SK_ID_CURR")

p_apps_curr = pd.DataFrame(index=application_train.index.tolist() + application_test.index.tolist())

p_apps_curr["hc_loan_num"] = (p_apps_mod_gr
                              .apply(lambda d:
                                     (d["NAME_CONTRACT_STATUS"] == "Approved")
                                     .sum()))

p_apps_curr["hc_loan_num_type"] = (p_apps_mod_gr
                                   .apply(lambda d:
                                          ((d["NAME_CONTRACT_STATUS"] ==
                                            "Approved") & (d["NAME_CONTRACT_TYPE"] ==
                                                              full_set.at[d["SK_ID_CURR"]
                                                                      .iloc[0],
                                                                      "NAME_CONTRACT_TYPE"]))
                                          .sum()))




p_apps_curr["hc_ref_num"] = (p_apps_mod_gr
                             .apply(lambda d:
                                    (d["NAME_CONTRACT_STATUS"] == "Refused")
                                    .sum()))

p_apps_curr["hc_ref_num_type"] = (p_apps_mod_gr
                                  .apply(lambda d:
                                         ((d["NAME_CONTRACT_STATUS"] ==
                                          "Refused") & (d["NAME_CONTRACT_TYPE"] ==
                                                             full_set.at[d["SK_ID_CURR"]
                                                                     .iloc[0],
                                                                     "NAME_CONTRACT_TYPE"]))
                                         .sum()))


p_apps_curr["hc_loan_ovd"] = (p_apps_mod_gr
                              .apply(lambda d:
                                     ((d["NAME_CONTRACT_STATUS"] == "Approved") &
                                      (d["is_overdued"]))
                                     .sum()))

p_apps_curr["hc_loan_ovd_type"] = (p_apps_mod_gr
                                   .apply(lambda d:
                                          ((d["NAME_CONTRACT_STATUS"] ==
                                            "Approved") & (d["is_overdued"]) &
                                           (d["NAME_CONTRACT_TYPE"] ==
                                            full_set.at[d["SK_ID_CURR"]
                                                    .iloc[0],
                                                    "NAME_CONTRACT_TYPE"]))
                                          .sum()))


p_apps_curr["hc_loan_amt"] = (p_apps_mod_gr
                              .apply(lambda d:
                                     d.loc[d["NAME_CONTRACT_STATUS"] == "Approved",
                                                "AMT_CREDIT"].sum()))

p_apps_curr["hc_loan_amt_type"] = (p_apps_mod_gr
                                   .apply(lambda d:
                                          d.loc[(d["NAME_CONTRACT_STATUS"] == "Approved") &
                                                (d["NAME_CONTRACT_TYPE"] ==
                                                 full_set.at[d["SK_ID_CURR"]
                                                         .iloc[0],
                                                         "NAME_CONTRACT_TYPE"]),
                                                "AMT_CREDIT"].sum()))


p_apps_curr["has_lim_exceeded"] = (p_apps_mod_gr
                                   .apply(lambda d:
                                          ((d["NAME_CONTRACT_STATUS"] ==
                                            "Approved") & (d["limit_exceeded"]))
                                          .sum()))

p_apps_curr.fillna(0,
                   axis=0,
                   inplace=True)


data = (application_train
        .join(bureau_curr)
        .join(p_apps_curr))

test = (application_test
        .join(bureau_curr)
        .join(p_apps_curr))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cols_to_impute_w_null = ["AMT_GOODS_PRICE",
                         "OBS_30_CNT_SOCIAL_CIRCLE",
                         "DEF_30_CNT_SOCIAL_CIRCLE",
                         "OBS_60_CNT_SOCIAL_CIRCLE",
                         "DEF_60_CNT_SOCIAL_CIRCLE",
                         "OWN_CAR_AGE",
                         "AMT_REQ_CREDIT_BUREAU_HOUR",
                         "AMT_REQ_CREDIT_BUREAU_DAY",
                         "AMT_REQ_CREDIT_BUREAU_WEEK",
                         "AMT_REQ_CREDIT_BUREAU_MON",
                         "AMT_REQ_CREDIT_BUREAU_QRT",
                         "AMT_REQ_CREDIT_BUREAU_YEAR"]

apartments_cols = data.columns[data.columns
                               .str.endswith(("_AVG", "_MODE", "_MEDI"))]

ext_source_cols = data.columns[data.columns
                               .str.startswith("EXT_")]


occu_gr_cols = ["NAME_INCOME_TYPE",
                "NAME_EDUCATION_TYPE",
                "CODE_GENDER",
                "ORGANIZATION_TYPE"]

exp_gr_cols = ["ORGANIZATION_TYPE", "OCCUPATION_TYPE"]

cols_to_discretize = ["AMT_INCOME_TOTAL",
                      "AMT_CREDIT",
                      "AMT_ANNUITY",
                      "AMT_GOODS_PRICE",
                      "loan_bal",
                      "hc_loan_amt",
                      "hc_loan_amt_type"]

cols_to_drop = (apartments_cols.tolist() +
                ext_source_cols.tolist() +
                ["EXP_GR_MAX", "EXP_GR_MIN"])

class SimpleColumnsAdder(BaseEstimator):

    def __init__(self, apartment_cols, cols_to_drop):
        self.apartment_cols = apartment_cols
        self.cols_to_drop = cols_to_drop


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **transform_params):
        X.loc[:, "APART_DESC_INTEGRITY"] = (1 - X[self.apartment_cols]
                                            .isnull().mean(axis=1))
        X.loc[:, "GOODS_RATIO"] = (X.apply(lambda row:
                                           0 if np.isnan(row["AMT_GOODS_PRICE"])
                                           else (row["AMT_GOODS_PRICE"] /
                                                 row["AMT_CREDIT"]), axis=1))
        X = X.eval("ANNUITY_RATIO = AMT_ANNUITY / AMT_INCOME_TOTAL")
        X.drop(self.cols_to_drop, axis=1, inplace=True)
        return X

class ExtSourceIntegrity(BaseEstimator):

    def __init__(self, ext_source_cols):
        self.ext_source_cols = ext_source_cols


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **transform_params):
        X.loc[:, "EXT_SOURCE_INTEGRITY"] = (X.apply(lambda row: 0
                                                    if (row[self.ext_source_cols]
                                                        .isnull().sum() == 3)
                                                    else (row[self.ext_source_cols]
                                                          .mean() *
                                                     (1 - row[self.ext_source_cols]
                                                      .isnull().mean())), axis=1))
        return X

class OccupationsImputer(BaseEstimator):

    def __init__(self, occu_gr_cols, occupations=None):
        self.occu_gr_cols = occu_gr_cols
        self.occupations = occupations


    def fit(self, X, y=None, **fit_params):
        self.occupations = (X
                            .groupby(self.occu_gr_cols)["OCCUPATION_TYPE"]
                            .apply(lambda d:
                                   d.value_counts(dropna=False).index[0])
                            .replace(np.nan, "Missed"))
        return self

    def transform(self, X, **transform_params):
        X.reset_index(inplace=True)
        X.set_index(self.occu_gr_cols, inplace=True)
        X.update(self.occupations)
        X.reset_index(inplace=True)
        X.set_index(["SK_ID_CURR"], inplace=True)
        return X

class DaysEmployedNormalizer(BaseEstimator):

    def __init__(self, exp_gr_cols, exp_gr_range=None):
        self.exp_gr_cols = exp_gr_cols
        self.exp_gr_range = exp_gr_range


    def fit(self, X, y=None, **fit_params):
        self.exp_gr_range = (X
                             .groupby(self.exp_gr_cols)["DAYS_EMPLOYED"]
                             .agg(["min", "max"]))
        self.exp_gr_range.columns = ["EXP_GR_MAX", "EXP_GR_MIN"]
        return self

    def transform(self, X, **transform_params):
        X = pd.merge(left=X,
                     right=self.exp_gr_range,
                     how="left",
                     left_on=self.exp_gr_cols,
                     right_index=True)
        X.loc[:, "DAYS_EMPLOYED_GR_NORM"] = X.apply(lambda row: 0
                                                    if (row["EXP_GR_MAX"] ==
                                                        row["EXP_GR_MIN"]) else
                                                    ((row["DAYS_EMPLOYED"] -
                                                      row["EXP_GR_MIN"]) /
                                                     (row["EXP_GR_MAX"] -
                                                      row["EXP_GR_MIN"])), axis=1)
        return X

class CustomOHEncoder(BaseEstimator):

    def __init__(self, cols=None):
        self.cols = cols


    def fit(self, X, y=None, **fit_params):
        self.cols = [re.compile(r"\[|\]|<", re.IGNORECASE).sub("_", col)
                     if any(x in str(col) for x in set(("[", "]", "<")))
                     else col for col in pd.get_dummies(X).columns.values]
        return self


    def transform(self, X, **transform_params):
        df = pd.get_dummies(X)
        for c in self.cols:
            if c not in df:
                df[c] = 0
        return df[self.cols]

class CustomImputer(BaseEstimator):

    def __init__(self, cols, strategy="constant", fill_value=None):
        self.strategy = strategy
        self.fill_value = fill_value
        self.cols = cols


    def fit(self, X, y=None, **fit_params):
        if self.strategy == "constant":
            self.fill_value = 0
        elif self.strategy == "mode":
            self.fill_value = X[self.cols].mode()[0]
        return self


    def transform(self, X, **transform_params):
        X.loc[:, self.cols] = X[self.cols].fillna(self.fill_value)
        return X

class CustomQuantileDiscretizer(BaseEstimator):

    def __init__(self, cols, col_bins=None, q=50):
        self.cols = cols
        self.col_bins = col_bins
        self.q = q


    def fit(self, X, y=None, **fit_params):
        self.col_bins = {}
        for col in self.cols:
            vals, bins = pd.qcut(x=X[col],
                                 q=self.q,
                                 duplicates="drop",
                                 precision=0,
                                 retbins=True)
            self.col_bins.update({col: bins})
        return self


    def transform(self, X, **transform_params):
        for col in self.cols:
            X.loc[:, col] = pd.cut(X[col], self.col_bins[col])
        return X



# Testing different Algorithms using above pipeline

In [ ]:
n_trees = 100
ada = AdaBoostClassifier(n_estimators=n_trees)
bag = BaggingClassifier(n_estimators=n_trees)
rf = RandomForestClassifier(n_estimators=n_trees)
et = ExtraTreesClassifier(n_estimators=n_trees)
gbm = GradientBoostingClassifier(n_estimators=n_trees)
target = data["TARGET"].value_counts()
spw = target[0] / target[1]
xgb_model = XGBClassifier(random_state=1234,
                          objective="binary:logistic",
                          scale_pos_weight=spw,                          
                          n_jobs=-1)
model_pipe = Pipeline(
    steps=[
        ("impute_nums", CustomImputer(strategy="constant",
                                         cols=cols_to_impute_w_null)),
        ("impute_cats",CustomImputer(strategy="mode",
                                        cols="NAME_TYPE_SUITE")),
        ("get_ext_source_integrity", ExtSourceIntegrity(ext_source_cols)),
        ("impute_occupations", OccupationsImputer(occu_gr_cols)),
        ("normalize_days_employed", DaysEmployedNormalizer(exp_gr_cols)),
        ("get_apart_desc_integrity", SimpleColumnsAdder(apartments_cols,
                                                           cols_to_drop)),
        ("dicretize_amt", CustomQuantileDiscretizer(cols_to_discretize)),
        ("oh_encoding", CustomOHEncoder()),
        ("xgb_model", xgb_model),
        ("ada", ada),
        ("bag", bag),
        ("rf", rf),
        ("et", et),
        ("gbm", gbm)
        ])

In [60]:
def evaluate_model(X, y, model, folds, metric):
	# create the pipeline
	pipeline = make_pipeline(model)
	# evaluate model
	scores = cross_val_score(pipeline, X, y, scoring=metric, cv=folds, n_jobs=-1)
	return scores